# 02: Hessian Matrix and Newton's Method

**Module 1.1: Calculus & Optimization**

## Learning Objectives

By the end of this notebook, you will:
1. Compute the Hessian matrix of second partial derivatives
2. Understand positive/negative definiteness and their geometric meaning
3. Classify critical points (minimum, maximum, saddle)
4. Implement Newton's method for optimization
5. Compare Newton's method to gradient descent

## Resources
- Solomon, *Numerical Algorithms*, §9.4.2
- Cohen, *Practical Linear Algebra*, Chapter 13

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd.functional import hessian

torch.manual_seed(42)
plt.rcParams['figure.figsize'] = (10, 6)

---
## 1. The Hessian Matrix

The **Hessian** is the matrix of all second partial derivatives:

$$H_f(\vec{x}) = \begin{bmatrix}
\frac{\partial^2 f}{\partial x_1^2} & \frac{\partial^2 f}{\partial x_1 \partial x_2} & \cdots & \frac{\partial^2 f}{\partial x_1 \partial x_n} \\
\frac{\partial^2 f}{\partial x_2 \partial x_1} & \frac{\partial^2 f}{\partial x_2^2} & \cdots & \frac{\partial^2 f}{\partial x_2 \partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial^2 f}{\partial x_n \partial x_1} & \frac{\partial^2 f}{\partial x_n \partial x_2} & \cdots & \frac{\partial^2 f}{\partial x_n^2}
\end{bmatrix}$$

### Key Properties

1. **Shape:** $n \times n$ for a function of $n$ variables
2. **Symmetric:** $H_{ij} = H_{ji}$ (mixed partials are equal)
3. **Diagonal:** Curvature in single variable directions
4. **Off-diagonal:** Variable interactions

In [ ]:
# Example: Compute Hessian by hand vs PyTorch

# f(x, y) = x² + 3xy + 2y²
# ∂f/∂x = 2x + 3y,  ∂f/∂y = 3x + 4y
# ∂²f/∂x² = 2,  ∂²f/∂y² = 4,  ∂²f/∂x∂y = 3

def f(xy):
    x, y = xy[0], xy[1]
    return x**2 + 3*x*y + 2*y**2

# Compute Hessian at any point (it's constant for this quadratic)
point = torch.tensor([1.0, 1.0])
H = hessian(f, point)

print("Function: f(x,y) = x² + 3xy + 2y²")
print("\nHessian (computed by PyTorch):")
print(H)

print("\nExpected (by hand):")
print("[[2, 3],")
print(" [3, 4]]")

---
## 2. Quadratic Form and Definiteness

The **quadratic form** $\vec{v}^\top H \vec{v}$ tells us about curvature in direction $\vec{v}$.

### Definiteness Classification

| Type | Condition | Geometric Meaning | At Critical Point |
|------|-----------|-------------------|-------------------|
| Positive definite | $\vec{v}^\top H \vec{v} > 0$ for all $\vec{v} \neq 0$ | Bowl (curves up) | **Minimum** |
| Negative definite | $\vec{v}^\top H \vec{v} < 0$ for all $\vec{v} \neq 0$ | Dome (curves down) | **Maximum** |
| Indefinite | Mixed signs | Saddle | **Saddle point** |

### 2×2 Shortcut

For $H = \begin{bmatrix} a & b \\ b & c \end{bmatrix}$:

- **Positive definite:** $a > 0$ AND $ac - b^2 > 0$
- **Negative definite:** $a < 0$ AND $ac - b^2 > 0$
- **Indefinite:** $ac - b^2 < 0$

In [ ]:
def check_definiteness(H):
    """Check definiteness of a 2x2 symmetric matrix."""
    a, b, c = H[0,0].item(), H[0,1].item(), H[1,1].item()
    det = a * c - b**2
    
    print(f"Matrix: [[{a}, {b}], [{b}, {c}]]")
    print(f"a = {a}, det = ac - b² = {det:.4f}")
    
    # Also check eigenvalues (general method)
    eigenvalues = torch.linalg.eigvalsh(H)
    print(f"Eigenvalues: {eigenvalues.tolist()}")
    
    if det > 0 and a > 0:
        return "Positive definite → MINIMUM"
    elif det > 0 and a < 0:
        return "Negative definite → MAXIMUM"
    elif det < 0:
        return "Indefinite → SADDLE POINT"
    else:
        return "Semidefinite (boundary case)"

# Test with our Hessian
H1 = torch.tensor([[2.0, 3.0], [3.0, 4.0]])
print("Example 1: f(x,y) = x² + 3xy + 2y²")
print(f"Result: {check_definiteness(H1)}")

print("\n" + "="*50 + "\n")

# A positive definite example
H2 = torch.tensor([[2.0, 0.0], [0.0, 4.0]])
print("Example 2: f(x,y) = x² + 2y² (simple bowl)")
print(f"Result: {check_definiteness(H2)}")

---
## 3. Visualizing Different Surface Types

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Create three functions: minimum, maximum, saddle
x = np.linspace(-2, 2, 50)
y = np.linspace(-2, 2, 50)
X, Y = np.meshgrid(x, y)

# Minimum (positive definite Hessian)
Z_min = X**2 + Y**2

# Maximum (negative definite Hessian)
Z_max = -(X**2 + Y**2)

# Saddle (indefinite Hessian)
Z_saddle = X**2 - Y**2

fig = plt.figure(figsize=(15, 5))

titles = ['Minimum\n$f = x^2 + y^2$\nH positive definite',
          'Maximum\n$f = -x^2 - y^2$\nH negative definite',
          'Saddle Point\n$f = x^2 - y^2$\nH indefinite']
surfaces = [Z_min, Z_max, Z_saddle]

for i, (Z, title) in enumerate(zip(surfaces, titles)):
    ax = fig.add_subplot(1, 3, i+1, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
    ax.scatter([0], [0], [0], color='red', s=100, label='Critical point')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('f(x,y)')
    ax.set_title(title)

plt.tight_layout()
plt.show()

---
## 4. Taylor Expansion (Multivariate)

The second-order Taylor expansion around $\vec{x}_k$:

$$f(\vec{x}) \approx f(\vec{x}_k) + \nabla f(\vec{x}_k)^\top (\vec{x} - \vec{x}_k) + \frac{1}{2}(\vec{x} - \vec{x}_k)^\top H_f(\vec{x}_k) (\vec{x} - \vec{x}_k)$$

### Breaking it down:

| Term | Meaning |
|------|--------|
| $f(\vec{x}_k)$ | Value at current point |
| $\nabla f^\top \Delta\vec{x}$ | First-order (linear) change |
| $\frac{1}{2}\Delta\vec{x}^\top H \Delta\vec{x}$ | Second-order (curvature) correction |

**Key insight:** For quadratic functions, this approximation is **exact**!

In [ ]:
# Demonstrate Taylor approximation
def f_nonquadratic(xy):
    x, y = xy[0], xy[1]
    return torch.sin(x) + torch.cos(y) + 0.1*x**2

def taylor_approximation(f, x0, dx):
    """Compute second-order Taylor approximation."""
    x0 = x0.clone().requires_grad_(True)
    
    # f(x0)
    f0 = f(x0)
    
    # Gradient
    f0.backward()
    grad = x0.grad.clone()
    
    # Hessian
    H = hessian(f, x0.detach())
    
    # Taylor approximation
    linear_term = torch.dot(grad, dx)
    quadratic_term = 0.5 * dx @ H @ dx
    
    return f0.item() + linear_term.item() + quadratic_term.item()

# Compare true vs approximation
x0 = torch.tensor([0.5, 0.5])
dx = torch.tensor([0.1, 0.1])

true_value = f_nonquadratic(x0 + dx).item()
approx_value = taylor_approximation(f_nonquadratic, x0, dx)

print(f"Point: x0 = {x0.tolist()}, step = {dx.tolist()}")
print(f"True f(x0 + dx): {true_value:.6f}")
print(f"Taylor approx:   {approx_value:.6f}")
print(f"Error:           {abs(true_value - approx_value):.6f}")

---
## 5. Newton's Method

### Derivation

1. Approximate $f$ with quadratic Taylor expansion
2. Set gradient of approximation to zero
3. Solve for the step $\Delta\vec{x}$

$$\nabla f + H\Delta\vec{x} = 0$$
$$\Delta\vec{x} = -H^{-1}\nabla f$$

### Update Rule

$$\vec{x}_{k+1} = \vec{x}_k - H_f^{-1}(\vec{x}_k) \nabla f(\vec{x}_k)$$

### Comparison with Gradient Descent

| | Gradient Descent | Newton's Method |
|---|-----------------|----------------|
| Update | $\vec{x} - \alpha \nabla f$ | $\vec{x} - H^{-1}\nabla f$ |
| Step size | Manual ($\alpha$) | Automatic ($H^{-1}$) |
| Convergence | Linear | Quadratic (near optimum) |
| Cost per step | $O(n)$ | $O(n^3)$ |
| Best for | Large-scale | Small-scale convex |

In [ ]:
def gradient_descent(f, x0, lr=0.1, max_iter=100, tol=1e-6):
    """Gradient descent optimization."""
    x = x0.clone()
    path = [x.clone().numpy()]
    
    for i in range(max_iter):
        x = x.requires_grad_(True)
        loss = f(x)
        loss.backward()
        grad = x.grad.clone()
        
        with torch.no_grad():
            x = x - lr * grad
        
        path.append(x.clone().numpy())
        
        if torch.norm(grad) < tol:
            break
    
    return x, np.array(path)


def newtons_method(f, x0, max_iter=100, tol=1e-6):
    """Newton's method optimization."""
    x = x0.clone()
    path = [x.clone().numpy()]
    
    for i in range(max_iter):
        x = x.requires_grad_(True)
        loss = f(x)
        loss.backward()
        grad = x.grad.clone()
        
        # Compute Hessian
        H = hessian(f, x.detach())
        
        # Newton step: solve H @ step = -grad
        step = torch.linalg.solve(H, -grad)
        
        with torch.no_grad():
            x = x + step
        
        path.append(x.clone().numpy())
        
        if torch.norm(grad) < tol:
            break
    
    return x, np.array(path)

In [ ]:
# Compare on an elongated bowl (harder for GD)
def elongated_bowl(xy):
    x, y = xy[0], xy[1]
    return x**2 + 10*y**2  # Very different curvature in x vs y

x0 = torch.tensor([5.0, 5.0])

# Run both methods
x_gd, path_gd = gradient_descent(elongated_bowl, x0, lr=0.05, max_iter=50)
x_newton, path_newton = newtons_method(elongated_bowl, x0, max_iter=10)

print(f"Gradient Descent: {len(path_gd)-1} iterations")
print(f"  Final point: ({path_gd[-1][0]:.6f}, {path_gd[-1][1]:.6f})")
print(f"\nNewton's Method: {len(path_newton)-1} iterations")
print(f"  Final point: ({path_newton[-1][0]:.6f}, {path_newton[-1][1]:.6f})")

In [ ]:
# Visualize the paths
x_range = np.linspace(-6, 6, 100)
y_range = np.linspace(-6, 6, 100)
X, Y = np.meshgrid(x_range, y_range)
Z = X**2 + 10*Y**2

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, path, title in [(axes[0], path_gd, 'Gradient Descent'),
                         (axes[1], path_newton, "Newton's Method")]:
    ax.contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.7)
    ax.plot(path[:, 0], path[:, 1], 'ro-', markersize=4, linewidth=1.5, label='Path')
    ax.plot(path[0, 0], path[0, 1], 'go', markersize=10, label='Start')
    ax.plot(path[-1, 0], path[-1, 1], 'b*', markersize=15, label='End')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f"{title}\n({len(path)-1} iterations)")
    ax.legend()
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 6. Why Newton Converges in 1 Step for Quadratics

For a **quadratic function**, the Taylor expansion is **exact** (no higher-order terms).

Therefore:
- Newton finds the exact minimum of the quadratic approximation
- Since approximation = true function, Newton finds the true minimum
- **One step is enough!**

In [ ]:
# Demonstrate: Newton converges in 1 step for quadratic
def quadratic(xy):
    x, y = xy[0], xy[1]
    return x**2 + 10*y**2 + 3*x + 5*y + 7  # Quadratic with linear terms

x0 = torch.tensor([10.0, 10.0])
x_result, path = newtons_method(quadratic, x0, max_iter=5)

print("Newton's method on quadratic function:")
for i, p in enumerate(path):
    f_val = quadratic(torch.tensor(p)).item()
    print(f"  Step {i}: x = ({p[0]:.6f}, {p[1]:.6f}), f(x) = {f_val:.6f}")

print("\n✓ Converged in 1 step!")

---
## 7. Genomics Application: When to Use Newton vs GD

| Tool | Method | Why |
|------|--------|-----|
| **DESeq2** | Newton-like (IRLS) | Few parameters per gene, convex GLM |
| **edgeR** | Newton-like | Same reasons |
| **scVI** | Adam (GD variant) | Millions of parameters, non-convex |
| **Neural networks** | SGD/Adam | Can't store/invert Hessian |

In [ ]:
# Simulating why Newton fails at scale

def memory_comparison(n_params):
    """Compare memory requirements."""
    gradient_memory = n_params * 4 / (1024**3)  # 4 bytes per float, convert to GB
    hessian_memory = n_params**2 * 4 / (1024**3)
    return gradient_memory, hessian_memory

print("Memory Requirements Comparison")
print("=" * 50)
print(f"{'Parameters':<15} {'Gradient':<15} {'Hessian':<15}")
print("-" * 50)

for n in [100, 1000, 10000, 100000, 1000000]:
    grad_mem, hess_mem = memory_comparison(n)
    grad_str = f"{grad_mem*1000:.2f} MB" if grad_mem < 1 else f"{grad_mem:.2f} GB"
    hess_str = f"{hess_mem*1000:.2f} MB" if hess_mem < 1 else f"{hess_mem:.2f} GB"
    if hess_mem > 1000:
        hess_str = f"{hess_mem/1000:.0f} TB ✗"
    print(f"{n:<15,} {grad_str:<15} {hess_str:<15}")

---
## Exercises

### Exercise 1: Hessian Computation

Compute the Hessian of $f(x, y) = e^x \cos(y)$ at the point $(0, 0)$.
Is it positive definite, negative definite, or indefinite?

### Exercise 2: Critical Point Classification

For $f(x, y) = x^3 - 3xy + y^3$:
1. Find all critical points (where $\nabla f = 0$)
2. Compute the Hessian at each critical point
3. Classify each as min/max/saddle

### Exercise 3: Newton vs GD Comparison

Create a function where gradient descent struggles but Newton succeeds quickly. Hint: Use very different curvatures in different directions.

### Exercise 4: DESeq2 Intuition

A negative binomial GLM has approximately 3 parameters per gene (intercept, treatment effect, dispersion). For 20,000 genes, how large would the full Hessian be if we optimized all genes jointly? Why does DESeq2 optimize genes independently instead?

In [ ]:
# Exercise 1: Your solution here



In [ ]:
# Exercise 2: Your solution here



---
## Summary

| Concept | Key Point |
|---------|----------|
| **Hessian** | Matrix of second derivatives, shape $(n \times n)$ |
| **Positive definite** | All eigenvalues > 0 → minimum |
| **Indefinite** | Mixed eigenvalues → saddle point |
| **Newton's method** | $\vec{x}_{k+1} = \vec{x}_k - H^{-1}\nabla f$ |
| **Newton vs GD** | Newton is faster but costs $O(n^3)$ |
| **Quadratic functions** | Newton converges in 1 step |

## Next Notebook

**03_jacobian_backprop.ipynb:** Jacobian matrices for vector-valued functions and backpropagation.